### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev cipher text, and unlabeled test cipher text, into lists.

In [1]:
train, dev, test = [], [], []

In [2]:
# from google.colab import files
# uploaded = files.upload()

Saving dev_enc.tsv to dev_enc.tsv
Saving test_enc_unlabeled.tsv to test_enc_unlabeled.tsv
Saving train_enc.tsv to train_enc.tsv


In [3]:
for x in open('./train_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    x[0] = int(x[0]) 
    train.append(x)
print (len(train))
print (train[:3])

16220
[[0, 'lkêcê yoúc cêêö y#êjl lw mówám Újám j Úêê# ütlk Úol lkêú z#ê ctöé8ú ówl xoóóú éê#xw#öê#c .'], [0, '6êcétlê jolêot8 zc éê#xw#öjóáê , tl zc j #jlkê# 8tcl8êcc jöÚ8ê 6wüó lkê öt668ê wx lkê #wj6 , ükê#ê lkê lkêöjltá t#wótêc j#ê lww wÚ2twoc jó6 lkê cê+oj8 éw8tltác lww cöoy .'], [0, 'tx lktc kw8t6jú öw2tê tc coééwcê6 lw Úê j ytxl , cwöêÚw6ú oóü#jééê6 tl êj#8ú , lwwm wol j88 lkê yww6 cloxx , jó6 8êxl Úêktó6 lkê á#jé ( 8tlê#j88ú ) .']]


In [4]:
for x in open('./dev_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    dev.append(x)
print (len(dev))
print (dev[:3])

2027
[[1, 'ów8jó Ú#j2ê8ú l#êj6c ükê#ê xêü jöê#tájó xt8öc 6j#ê lw 6ê82ê 77 tólw lkê üw#86 wx jöÚt2j8êóáê jó6 jöÚtyotlú <<<'], [0, 'ê2êó öo#ékú zc ê+éê#l áwötá ltötóy jó6 xjöê6 ákj#tcöj áj ózl #êcáoê lktc êxxw#l .'], [1, 'üt88 jcco#ê68ú #jóm jc wóê wx lkê á8ê2ê#êcl , öwcl 6êáêélt2ê8ú jöoctóy áwöê6têc wx lkê úêj# .']]


#### Different from 'train' and 'dev' that are both list of tuples, 'test' will be just a list.

In [5]:
for x in open('./test_enc_unlabeled.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r')
    test.append(x)
print (len(test))
print (test[:3])

2028
['j 6t6jáltá jó6 6o88 6wáoöêólj#ú y8w#txútóy cwxlüj#ê jój#ákú .', 'ówlktóy cltámc , #êj88ú , ê+áêél j 8tóyê#tóy á#êêétóêcc wóê xêê8c x#wö Úêtóy 6#jyyê6 lk#woyk j cj6 , cw#6t6 oót2ê#cê wx yoóc , 6#oyc , j2j#táê jó6 6jöjyê6 6#êjöc .', 'öo#ékú jó6 üt8cwó jáloj88ú öjmê j é#êllú yww6 lêjö <<< Úol lkê é#wvêál co##woó6tóy lkêö tc 6tcl#êcctóy8ú #wlê .']


#### You can split every sentence into lists of words by white spaces.

In [6]:
train_split = [[x[0], x[1].split(' ')] for x in train]
dev_split = [[x[0], x[1].split(' ')] for x in dev]
test_split = [[x.split(' ')] for x in test]

### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

In [7]:
# Eventually, results need to be a list of 2028 0 or 1's
results = []

In [8]:
import sklearn
import numpy as np

In [9]:
train_split = np.array(train)
dev_split = np.array(dev)
train_l, train_d = train_split[:, 0], train_split[:, 1]
dev_l, dev_d = dev_split[:, 0], dev_split[:, 1]

In [10]:
train_d = list(train_d)
train_l = list(train_l)

dev_d = list(dev_d)
dev_l = list(dev_l)

In [11]:
clean_train = []
clean_test = []
import string
for s in train_d:
  t = s.translate(str.maketrans('', '', string.punctuation))
  clean_train.append(t)

for s in dev_d:
  t = s.translate(str.maketrans('', '', string.punctuation))
  clean_test.append(t)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_d)
X_train_counts.shape

(16220, 17248)

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(16220, 17248)

In [14]:
X_dev_counts = count_vect.fit_transform(dev_d)
X_dev_counts.shape
X_dev_tfidf = tfidf_transformer.fit_transform(X_dev_counts)
X_dev_tfidf.shape

(2027, 6790)

In [15]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train_l)

In [16]:
from sklearn.pipeline import Pipeline

In [17]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [18]:
text_clf = text_clf.fit(train_d, train_l)

In [19]:
import numpy as np
predicted = text_clf.predict(dev_d)
np.mean(predicted == dev_l)

0.852491366551554

In [20]:
import numpy as np
results = text_clf.predict(test)
print(results[0:5], results[-5:])

['0' '0' '0' '0' '0'] ['1' '1' '1' '1' '1']


In [49]:
print(len(test))

2028


In [23]:
count_one = 0
count_zero = 0
for i in predicted:
  if i == '0':
    count_zero += 1
  else:
    count_one += 1
print(count_zero, count_one)

1017 1011


Trying SVM Classifier now

---



In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42))])

In [ ]:
text_clf_svm = text_clf_svm.fit(train_d, train_l)

In [ ]:
predicted_svm = text_clf_svm.predict(dev_d)

In [ ]:
np.mean(predicted_svm == dev_l)

0.7567834237789837

Grid Search

---



In [26]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [27]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(train_d, train_l)

FFNN

In [50]:
seed = 7
np.random.seed(seed)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [21]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 2028)

In [22]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [23]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')